# Assignment 1 — N-Puzzle Project Code and Documentation

This project was originally implemented as a collection of python scripts/modules as can be seen on https://github.com/antonnemch/CP468_Projects/
It was compiled into this notebook for ease of submission, with one module per code cell. An overview of the documentation is found in the table of contents below, and more details can be found in the headers of specific files.

## Table of Contents & Documentation

1. requirements.txt  
2. puzzle.py — Board implementation  
3. heuristics.py — Heuristic functions (h₁–h₃)  
4. search.py — A* search algorithm  
5. metrics.py — Branching factor utilities  
6. run_experiments.py — Experiment driver & visualization

---

## 1. requirements.txt

**Required packages (brief):**

- `matplotlib` — for plotting results and visualizations  
- `numpy` — numeric helpers for heuristics and metrics  
- `pandas` — tabular data, CSV output, experiment summaries  
- `pytest` — testing framework for module correctness

---

## 2. puzzle.py

> **Author:** Anton Nemchinski

This module implements an **immutable**, **memory-efficient** representation of the sliding-tile n-puzzle and a small set of helpers used by the experiment harness.

### Key Design Points
- `Board` is a **frozen dataclass** with `slots=True` → immutable, hashable, lightweight.  
- Tiles are stored as a **flat 1-D tuple** of length `n*n`, with `0` representing the blank.  
- The blank index is cached in `zero` for efficient neighbor generation.  
- All methods return **new `Board` instances** (no mutation).

### API (At-a-Glance)
- **`class Board(n, tiles, zero)`**
  - `Board.goal(n)` → solved board  
  - `Board.from_flat(n, flat)` → build board from sequence (validates input)  
  - `Board.from_rows(rows)` → build board from nested lists  
  - `Board.is_goal()` → check if solved  
  - `Board.is_solvable()` → check solvability via inversion counts  
  - `Board.visualize()` → ASCII multi-line view  
  - `Board.neighbors()` → generate legal neighbor states (up to 4)  
  - `Board.randomize(k, seed=None)` → produce reachable randomized board

- **Helpers**
  - `manual_move(board, direction)` → apply single U/D/L/R move  
  - `manual_play(n=3, r=10)` → small interactive puzzle console

---

## 3. heuristics.py

> **Author:** Julian Rincon

Implements admissible heuristics for A*:

- **h₁ Misplaced Tiles:** Number of misplaced non-blank tiles  
- **h₂ Manhattan Distance:** Sum of Manhattan distances to goal positions  
- **h₃ Linear Conflict:** Manhattan + 2×(linear conflicts), stronger but still admissible

### API (At-a-Glance)
- `_goal_pos_map(n)` → dict mapping tile → goal `(row, col)`  
- `h1_misplaced(board)` → int  
- `h2_manhattan(board)` → int  
- `h3_linear_conflict(board)` → int

---

## 4. search.py

> **Authors:** Roop Bassi, Jordan Franschman

Straightforward **A\*** search implementation. Focuses on clarity and correctness.

- Stores `g`-values and parents for path reconstruction  
- Uses a **heap** keyed by `f = g + h`  
- Returns a `SearchResult` dataclass

### API (At-a-Glance)
- `SearchResult(solved, solution_depth, nodes_expanded, runtime, path)`  
- `astar(board, heuristic)` → `SearchResult`

> Includes a small demo when run as a script (random 3×3 board).

---

## 5. metrics.py

> **Author:** Jordan Franschman

Small utility module for **branching factor analysis**.

### API (At-a-Glance)
- `branching_factor(nodes_expanded, solution_depth, tolerance=0.01)` → float  
  Solves for **b\*** using binary search:
  \[
  1 + b^* + (b^*)^2 + \dots + (b^*)^d = \text{nodes\_expanded}
  \]

Returns `0.0` for trivial cases.

---

## 6. run_experiments.py

> **Author:** Anton Nemchinski

Experiment driver & visualization utilities.

- Runs A* on randomized puzzles with multiple heuristics  
- Collects & summarizes results using pandas  
- Generates comparison tables, CSVs, and plots

### API (At-a-Glance)
- `run_single_experiment(n, num_moves, seed, h_funcs)`  
- `run_multiple_experiments(n, num_moves, num_experiments, h_funcs)`  
- `calculate_statistics(results)` → DataFrame  
- `create_and_save_comparison_table(...)` → CSV + DataFrame  
- `save_image_table_nodes_and_bf(...)` → PNG table & CSV  
- `save_comparison_graphs(...)` → scatter/line plots  
- `ensure_figures_directory(n)` → directory path  
- `run_part_1/2/3()` → run experiments for 8-, 15-, and 24-puzzle

---


## 1. requirements.txt

In [ ]:
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install pytest

## 2. puzzle.py

In [ ]:
"""n-puzzle Board implementation and utilities.

This module implements an immutable, memory-efficient representation of the n-puzzle
(sliding tile puzzle) and a small set of helpers used by the experimental harness.

Author
------
Anton Nemchinski

Key design points
-----------------
- Board instances are immutable (a frozen dataclass with ``slots=True``). This makes
    them hashable and cheap to copy by reference when used as keys in dictionaries or
    sets (useful for search algorithms).
- The board is stored as a flat 1-D tuple ``tiles`` of length ``n*n`` where the
    blank tile is represented by 0. The index of the blank is cached in ``zero`` for
    fast neighbor generation.
- Methods return new ``Board`` objects instead of mutating state. The module
    provides convenience constructors and utilities for testing and interactive use.

API (at-a-glance)
------------------
- class Board(n, tiles, zero):
    - Board.goal(n) -> Board
        Return solved board for size n. O(n^2) time to construct.
    - Board.from_flat(n, flat) -> Board
        Validate and return board from flat sequence. O(n^2).
    - Board.from_rows(rows) -> Board
        Build board from nested list.
    - Board.is_goal() -> bool
        True if board equals goal state. O(1) tuple comparison.
    - Board.is_solvable() -> bool
        Determine solvability using inversion counts (standard rule). O(n^2).
    - Board.visualize() -> str
        Return an ASCII multi-line representation for printing. O(n^2).
    - Board.neighbors() -> Iterator[Board]
        Yield neighbor boards by sliding the blank. Generates up to 4 neighbors. O(n^2)
        per neighbor creation (cost copies the tile tuple).
    - Board.randomize(k, seed=None) -> Board
        Produce a board reachable from this one after k random legal moves.

- module-level helpers:
    - manual_move(board, direction) -> Board
        Apply a single move 'U','D','L','R' to the given board (raises if illegal).
    - manual_play(n=3, r=10)
        Small interactive console to play the puzzle (helpful for manual testing).

Examples

        >>> b = Board.goal(3)
        >>> print(b.visualize())
         1  2  3
         4  5  6
         7  8  X

        # randomize 10 legal moves from the goal (deterministic with seed)
        >>> r = b.randomize(10, seed=42)

Notes
-----
- All random scrambles generated by :meth:`randomize` are reachable from the goal
    (and therefore solvable) because they are produced by applying legal moves to
    the goal state.
- The :meth:`is_solvable` method implements standard inversion-count rules for
    determining solvability.

"""

from __future__ import annotations
from dataclasses import dataclass
from typing import Iterator, Tuple, List, Optional, Sequence
import random

# Main Board Class
@dataclass(frozen=True, slots=True)
class Board:
    """
    Immutable n-puzzle board.
    tiles: flattened tuple of length n*n with 0 as the blank.
    zero: index of the blank in tiles (cached for speed).
    """
    n: int
    tiles: Tuple[int, ...]
    zero: int

    # Constructors ####################################################
    @staticmethod
    def goal(n: int) -> "Board":
        """Return the solved n×n board: (1..n*n-1,0)."""
        flat = tuple(range(1, n * n)) + (0,)
        return Board(n, flat, flat.index(0))
    
    @staticmethod
    def from_flat(n: int, flat: Sequence[int]) -> "Board":
        """Build a board from a flat sequence of length n*n."""
        flat = tuple(flat)
        if len(flat) != n*n:
            raise ValueError(...)
        if set(flat) != set(range(n*n)):
            raise ValueError(...)
        return Board(n, flat, flat.index(0))

    @staticmethod
    def from_rows(rows: List[List[int]]) -> "Board":
        """Build a board from a square list-of-lists. Validates shape and contents."""
        n = len(rows)
        flat = tuple(x for row in rows for x in row)
        return Board.from_flat(n, flat)
    
    # Methods #########################################################
    def is_goal(self) -> bool:
        """True if this board is the solved state."""
        # Compare tuples directly (fast).
        return self.tiles == Board.goal(self.n).tiles

    def is_solvable(self) -> bool:
        """
        Calculates disorder parameter (inversions) to determine if the board is solvable.
        - Odd n: solvable iff inversions even.
        - Even n: solvable iff (inversions + blank_row_from_bottom) is odd.
        """
        n = self.n
        arr = [x for x in self.tiles if x != 0]
        inv = 0
        for i in range(len(arr)):
            ai = arr[i]
            inv += sum(1 for j in range(i + 1, len(arr)) if arr[j] < ai)

        if n % 2 == 1:
            return inv % 2 == 0
        else:
            row_from_bottom = n - (self.zero // n)  # 1-based
            return (inv + row_from_bottom) % 2 == 1

    def visualize(self) -> str:
        """ASCII grid; blank as spaces."""
        n = self.n
        rows = [self.tiles[i : i + n] for i in range(0, n * n, n)]
        return "\n".join(
            " ".join(f"{x:2d}" if x != 0 else " X" for x in row) for row in rows
        )


    def neighbors(self) -> Iterator["Board"]:
        """
        Yield all legal boards by sliding the blank up/down/left/right.
        Deterministic order: Up, Down, Left, Right.
        """
        n, z = self.n, self.zero
        r, c = divmod(z, n)
        deltas = []
        if r > 0:     deltas.append(-n)  # Up
        if r < n - 1: deltas.append(+n)  # Down
        if c > 0:     deltas.append(-1)  # Left
        if c < n - 1: deltas.append(+1)  # Right

        t = self.tiles
        for d in deltas:
            nz = z + d
            lst = list(t)
            lst[z], lst[nz] = lst[nz], lst[z]
            yield Board(n, tuple(lst), nz)


    # Utilities #######################################################
    def randomize(self, k: int, seed: Optional[int] = None) -> "Board":
        """
        Return a new board reached by k random legal blank moves.
        Avoids immediate backtracking when possible. Always solvable.
        """
        rng = random.Random(seed)
        b = self
        prev_tiles: Tuple[int, ...] | None = None
        for _ in range(k):
            nbrs = list(b.neighbors())
            if prev_tiles is not None and len(nbrs) > 1:
                nbrs = [x for x in nbrs if x.tiles != prev_tiles]
            prev_tiles = b.tiles
            b = rng.choice(nbrs)
        return b
    
def manual_move(b: Board, direction: str) -> Board:
    """
    Return a new board by sliding the blank in the given direction.
    Direction is one of "U", "D", "L", "R" (case-insensitive).
    Raises ValueError if the move is illegal.
    """
    direction = direction.upper()
    n, z = b.n, b.zero
    r, c = divmod(z, n)
    if direction == "U":
        if r == 0:
            raise ValueError("Illegal move Up")
        d = -n
    elif direction == "D":
        if r == n - 1:
            raise ValueError("Illegal move Down")
        d = +n
    elif direction == "L":
        if c == 0:
            raise ValueError("Illegal move Left")
        d = -1
    elif direction == "R":
        if c == n - 1:
            raise ValueError("Illegal move Right")
        d = +1
    else:
        raise ValueError("Direction must be one of U,D,L,R")

    nz = z + d
    lst = list(b.tiles)
    lst[z], lst[nz] = lst[nz], lst[z]
    return Board(n, tuple(lst), nz)

def manual_play(n: int = 3,r: int = 10) -> None:
    """
    Simple interactive console play of the puzzle.
    Commands: U,D,L,R to move; Q to quit; R to randomize; H for help.
    """
    print("Welcome to the n-puzzle! Commands: U,D,L,R to move; Q to quit; R to randomize; H for help.")
    board = Board.goal(n).randomize(r, seed=42)
    while True:
        print("\nCurrent board:")
        print(board.visualize())
        if board.is_goal():
            print("Congratulations! You've solved the puzzle!")
            break
        cmd = input("Enter command (U/D/L/R/Q/R/H): ").strip().upper()
        if cmd == "Q":
            print("Thanks for playing!")
            break
        elif cmd == "H":
            print("Commands: U,D,L,R to move; Q to quit; R to randomize; H for help.")
        elif cmd == "R":
            board = Board.goal(n).randomize(10)
            print("Board randomized.")
        elif cmd in ("U", "D", "L", "R"):
            try:
                board = manual_move(board, cmd)
            except ValueError as e:
                print(e)
        else:
            print("Invalid command. Type H for help.")

## 3. heuristics.py

In [ ]:
"""Heuristic functions for the n-puzzle.

This module provides admissible heuristics used by the A* search driver:
- h1_misplaced: number of misplaced tiles (ignore blank)
- h2_manhattan: sum of Manhattan distances to goal positions
- h3_linear_conflict: Manhattan + 2 * (linear conflict pairs)

Author
------
Julian Rincon

API (at-a-glance)
------------------
- _goal_pos_map(n) -> Dict[int, (row, col)]
    Map tile value to its goal coordinates (excludes blank).
- h1_misplaced(board: Board) -> int
    Count of non-blank tiles not in goal position. O(n^2).
- h2_manhattan(board: Board) -> int
    Sum of Manhattan distances for all tiles. O(n^2).
- h3_linear_conflict(board: Board) -> int
    Manhattan plus linear conflict correction; admissible and
    typically stronger than Manhattan. O(n^2) with a small constant.

"""

from __future__ import annotations
from typing import Tuple, Dict
# from puzzle import Board

def _goal_pos_map(n: int) -> Dict[int, Tuple[int, int]]:
    """
    Map each tile value -> (goal_row, goal_col).
    Tile 0 (blank) is excluded.
    Goal layout is (1..n*n-1, 0).
    """
    m: Dict[int, Tuple[int, int]] = {}
    for v in range(1, n * n):
        r, c = divmod(v - 1, n)
        m[v] = (r, c)
    return m

def h1_misplaced(board: Board) -> int:
    """Number of tiles not in their goal position (ignore blank)."""
    return sum(
        1
        for i, v in enumerate(board.tiles)
        if v != 0 and i != (v - 1)
    )

def h2_manhattan(board: Board) -> int:
    """Sum of Manhattan distances of each tile from its goal position."""
    n = board.n
    goal = _goal_pos_map(n)
    dist = 0
    for i, v in enumerate(board.tiles):
        if v == 0:
            continue
        r, c = divmod(i, n)
        gr, gc = goal[v]
        dist += abs(r - gr) + abs(c - gc)
    return dist
# Helper function for h3
def _linear_conflicts_in_line(line_vals, line_index, is_row, goal):
    """
    Count linear conflicts among tiles in one row/column.
    - Only tiles whose goal row (or column) is this line are considered.
    - A pair (a,b) is in conflict if both belong to this line in goal,
      but their order is reversed relative to goal columns (or rows).
    Returns the count of *conflicting pairs* (each adds +2 to heuristic).
    """
    conflicts = 0
    seq = []
    for idx_along, v in enumerate(line_vals):
        if v == 0:
            continue
        gr, gc = goal[v]
        if is_row:
            if gr == line_index:
                seq.append((idx_along, gc))
        else:
            if gc == line_index:
                seq.append((idx_along, gr))
    
    for i in range(len(seq)):
        for j in range(i + 1, len(seq)):
            if seq[i][1] > seq[j][1]:
                conflicts += 1
    return conflicts

def h3_linear_conflict(board: Board) -> int:
    """
    Linear Conflict heuristic:
      h = Manhattan + 2 * (# of linear-conflict pairs in rows and columns)
    Admissible and consistent; stronger than pure Manhattan.
    """
    n = board.n
    goal = _goal_pos_map(n)
    man = h2_manhattan(board)

    # Row 
    row_conflicts = 0
    for r in range(n):
        start = r * n
        row_vals = board.tiles[start : start + n]
        row_conflicts += _linear_conflicts_in_line(row_vals, r, True, goal)

    # Column
    col_conflicts = 0
    for c in range(n):
        col_vals = board.tiles[c : n * n : n]
        col_conflicts += _linear_conflicts_in_line(col_vals, c, False, goal)

    return man + 2 * (row_conflicts + col_conflicts)


## 4. search.py

In [ ]:
"""A* search implementation and a simple SearchResult container.

This module provides a straightforward A* implementation used by the
experiment harness. It focuses on clarity and correctness rather than
performance micro-optimizations. The A* implementation stores g-values,
parents for path reconstruction, and uses a heap keyed by f = g + h.

Authors
-------
Roop Bassi
Jordan Franschman

API (at-a-glance)
------------------
- dataclass SearchResult(solved, solution_depth, nodes_expanded, runtime, path)
    Container returned by :func:`astar` describing the outcome.
- astar(board, heuristic) -> SearchResult
    Run A* from the given start board using the provided heuristic function
    (callable taking a Board and returning an int). Returns a SearchResult.

Note: This module includes a small demo that runs A* on a randomized 3x3
board when executed as a script. Remove or guard demo code if embedding in
larger systems.

"""

#import heuristics, puzzle
from heapq import heappush, heappop
from itertools import count
import time
from dataclasses import dataclass
from typing import Optional, List

@dataclass
class SearchResult:
    solved: bool
    solution_depth: int
    nodes_expanded: int
    runtime: float
    path: Optional[List] = None

def astar(board, heuristic):
    # record start time
    start_time = time.perf_counter()

    heap = []
    nodes_expanded = 0
    g_values = {board.tiles: 0}
    parent = {board.tiles: None}  #track parents for path
    tie_counter = count() # need this in case of a tie so it doesn't check next element

    # Push f(n) = h(n)+0, g(n), tie_counter, board
    heappush(heap, (heuristic(board), 0, next(tie_counter), board))

    while heap:
        f, g, tie, board = heappop(heap)

        if g_values.get(board.tiles, 1000000000000000) < g:
            continue

        nodes_expanded += 1

        if board.is_goal():
            path = []
            current = board
            while current is not None:
                path.append(current)
                current = parent.get(current.tiles)
            path.reverse()
            
            runtime = time.perf_counter() - start_time
            return SearchResult(True, g, nodes_expanded, runtime, path)

        for neighbour in board.neighbors():
            total_g = g + 1
            if total_g < g_values.get(neighbour.tiles, 1000000000000000):
                g_values[neighbour.tiles] = total_g
                parent[neighbour.tiles] = board  # Track parent
                fn = total_g + heuristic(neighbour)
                heappush(heap, (fn, total_g, next(tie_counter), neighbour))
                
    runtime = time.perf_counter()-start_time  # calculate runtime
    return SearchResult(False, -1, nodes_expanded, runtime, None)  # CHANGE THIS


board = Board.goal(3).randomize(15, seed=42)
print(board)

res_h1 = astar(board, h1_misplaced)
print(f'Solved: {res_h1.solved}\nTotal Steps:{res_h1.solution_depth}\nNodes Expanded: {res_h1.nodes_expanded}\nRuntime: {res_h1.runtime:.4f}s\n')

res_h2 = astar(board, h2_manhattan)
print(f'Solved: {res_h2.solved}\nTotal Steps:{res_h2.solution_depth}\nNodes Expanded: {res_h2.nodes_expanded}\nRuntime: {res_h2.runtime:.4f}s\n')

res_h3 = astar(board, h3_linear_conflict)
print(f'Solved: {res_h3.solved}\nTotal Steps:{res_h3.solution_depth}\nNodes Expanded: {res_h3.nodes_expanded}\nRuntime: {res_h3.runtime:.4f}s\n')


## 5. metrics.py

In [ ]:
"""Metrics utilities for analyzing search behavior.

Currently this module exposes a single utility used by the experiment
driver to compute the effective branching factor (b*), the branching
factor that would produce the observed number of expanded nodes at a
given solution depth in a uniform tree.

Author
------
Jordan Franschman

API (at-a-glance)
------------------
- branching_factor(nodes_expanded: int, solution_depth: int, tolerance: float = 0.01) -> float
  Numerically solve for b* in the equation: 1 + b* + b*^2 + ... + b*^d = nodes_expanded
  using binary search. Returns 0.0 for trivial cases.

"""

def branching_factor(nodes_expanded: int, solution_depth: int, tolerance: float = 0.01) -> float:

    if solution_depth == 0 or nodes_expanded <= 1:
        return 0.0
    
    low,high = 1.0, float(nodes_expanded)
    
    while high-low > tolerance:
        mid = (low + high) / 2.0
      
        if abs(mid - 1.0) < 1e-9:
            total = solution_depth + 1
        else:
            total = (mid**(solution_depth + 1) - 1) / (mid - 1)
        
        if total < nodes_expanded:
            low = mid
        else:
            high = mid
    
    return (low + high) / 2.0


## 6. run_experiments.py

In [ ]:
"""Experiment driver and visualization utilities for the n-puzzle project.

This module orchestrates experiments that run A* with multiple heuristics
across different puzzle sizes, collects results, computes summaries, and
produces CSVs and figures organized under a Results/PartX directory layout.

Author
------
Anton Nemchinski

API (at-a-glance)
------------------
- run_single_experiment(n, num_moves, seed, h_funcs) -> Dict[str, SearchResult]
    Run A* on a single randomized instance for all heuristics provided.
- run_multiple_experiments(n, num_moves, num_experiments, h_funcs)
    Produce results for many instances; prints progress and returns a dict
    mapping heuristic name -> list of SearchResult.
- calculate_statistics(results) -> pd.DataFrame
    Aggregate per-heuristic statistics (mean nodes, depth, runtime, b*, success_rate).
- create_and_save_comparison_table(results, n, base_path) -> pd.DataFrame
    Save a CSV comparing nodes expanded and branching factor per solution depth.
- save_image_table_nodes_and_bf(results, n, base_path) -> pd.DataFrame
    Produce the PNG table visual used in reports and a CSV summary.
- save_comparison_graphs(results, n, base_path)
    Save scatter and line plots (nodes vs depth, branching factor vs depth).
- ensure_figures_directory(n) -> str
    Create Results/PartX directories and return the path appropriate for n.
- run_part_1/2/3()
    Runners for 8-, 15-, and 24-puzzle experiments respectively. Each returns
    the raw results dict for that part.

"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from typing import List, Dict, Callable


# Project modules
# from puzzle import Board
# from search import astar, SearchResult
# import heuristics
# from metrics import branching_factor


def save_image_table_nodes_and_bf(results: Dict[str, List[SearchResult]], n: int, base_path: str):
    """Generate and save an image table like the textbook, for nodes and branching factor by depth and heuristic."""
    depth_groups = defaultdict(lambda: defaultdict(list))
    for h_name, h_results in results.items():
        for result in h_results:
            if result.solved:
                depth_groups[result.solution_depth][h_name].append(result)
    heuristics = list(results.keys())
    table_data = []
    for d in sorted(depth_groups.keys()):
        row = [d]
        # Nodes expanded for each heuristic
        for h_name in heuristics:
            vals = [r.nodes_expanded for r in depth_groups[d].get(h_name, [])]
            row.append(int(np.mean(vals)) if vals else '')
        # Branching factor for each heuristic
        for h_name in heuristics:
            vals = [branching_factor(r.nodes_expanded, r.solution_depth) for r in depth_groups[d].get(h_name, [])]
            row.append(round(np.mean(vals), 2) if vals else '')
        table_data.append(row)
    # Build column headers
    col_labels = ["d"] + [f"{h} Nodes" for h in heuristics] + [f"{h} BF" for h in heuristics]
    # Adjust figure size to fit table tightly
    fig_height = 0.6 * len(table_data) + 1.8
    fig_width = max(2 + 2 * len(heuristics), 8)
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    ax.axis('off')
    tbl = ax.table(cellText=table_data, colLabels=col_labels, loc='center', cellLoc='center')
    for col_idx in range(len(col_labels)):
        tbl.auto_set_column_width(col=col_idx)
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(11)
    tbl.scale(1.1, 1.1)
    # Style header row
    for (row, col), cell in tbl.get_celld().items():
        if row == 0:
            cell.set_fontsize(12)
            cell.set_text_props(weight='bold')
            cell.set_facecolor('#e0e0e0')
        # Remove cell borders for a cleaner look
        cell.set_linewidth(0.5)
    # Center the table title and reduce whitespace
    plt.tight_layout()
    plt.subplots_adjust(left=0.08, right=0.92, top=0.82, bottom=0.08)
    plt.title(f"A* Search Cost and Branching Factor ({n}x{n} Puzzle)", fontsize=14, pad=10)
    plt.savefig(f"{base_path}/table_nodes_bf_{n}x{n}.png", bbox_inches='tight', dpi=200)
    plt.close()
    # import matplotlib.ticker as ticker
    depth_groups = defaultdict(lambda: defaultdict(list))
    for h_name, h_results in results.items():
        for result in h_results:
            if result.solved:
                depth_groups[result.solution_depth][h_name].append(result.nodes_expanded)
    table_data = []
    heuristics = list(results.keys())
    for d in sorted(depth_groups.keys()):
        row = {'d': d}
        for h_name in heuristics:
            vals = depth_groups[d].get(h_name, [])
            row[h_name] = int(np.mean(vals)) if vals else ''
        table_data.append(row)
    df = pd.DataFrame(table_data)
    df.to_csv(f'{base_path}/summary_nodes_by_depth_{n}x{n}.csv', index=False)
    return df

def run_single_experiment(n: int, num_moves: int, seed: int, 
                         h_funcs: Dict[str, Callable]) -> Dict[str, SearchResult]:
    """Run experiment for a single puzzle instance with all heuristics."""
    board = Board.goal(n).randomize(num_moves, seed=seed)
    results = {}
    
    for h_name, h_func in h_funcs.items():
        result = astar(board, h_func)
        results[h_name] = result
        
    return results

def run_multiple_experiments(n: int, num_moves: int, num_experiments: int,
                           h_funcs: Dict[str, Callable]) -> Dict[str, List[SearchResult]]:
    """Run multiple experiments and collect results."""
    results = defaultdict(list)
    
    for seed in range(num_experiments):
        single_results = run_single_experiment(n, num_moves, seed, h_funcs)
        for h_name, result in single_results.items():
            results[h_name].append(result)
        if (seed + 1) % 10 == 0 or (seed + 1) == num_experiments:
            print(f"  Completed {seed + 1}/{num_experiments} experiments...")
    return dict(results)

def calculate_statistics(results: Dict[str, List[SearchResult]]) -> pd.DataFrame:
    """Calculate statistics from experimental results."""
    stats = {}
    
    for h_name, h_results in results.items():
        solved_results = [r for r in h_results if r.solved]
        if not solved_results:
            continue
            
        stats[h_name] = {
            'solution_depth_mean': int(np.mean([r.solution_depth for r in solved_results])),
            'nodes_expanded_mean': int(np.mean([r.nodes_expanded for r in solved_results])),
            'runtime_mean': np.mean([r.runtime for r in solved_results]).round(4),
            'branching_factor': np.mean([
                branching_factor(r.nodes_expanded, r.solution_depth) 
                for r in solved_results
            ]).round(3),
            'success_rate': round(len(solved_results) / len(h_results), 3)
        }

    return pd.DataFrame(stats).round(3)

def create_and_save_comparison_table(results: Dict[str, List[SearchResult]], n: int, base_path: str) -> pd.DataFrame:
    """Create and save a detailed comparison table."""
    depth_groups = defaultdict(lambda: defaultdict(list))
    
    for h_name, h_results in results.items():
        for result in h_results:
            if result.solved:
                depth_groups[result.solution_depth][h_name].append(result)
    
    table_data = []
    for d in sorted(depth_groups.keys()):
        row = {'Solution Depth': d}
        for h_name, results_at_depth in depth_groups[d].items():
            nodes = np.mean([r.nodes_expanded for r in results_at_depth])
            bf = np.mean([
                branching_factor(r.nodes_expanded, r.solution_depth)
                for r in results_at_depth
            ])
            row[f'{h_name} Nodes Expanded'] = int(nodes)
            row[f'{h_name} Branching Factor'] = round(bf, 2)
        table_data.append(row)
    
    df = pd.DataFrame(table_data)
    # Save as CSV only
    df.to_csv(f'{base_path}/comparison_table_{n}x{n}.csv', index=False)
    return df

def analyze_heuristic_domination(results: Dict[str, List[SearchResult]]) -> pd.DataFrame:
    """
    Analyze whether heuristics dominate each other.
    A heuristic h1 dominates h2 if:
    1. h1 expands fewer or equal nodes for all problems
    2. h1 expands strictly fewer nodes for at least one problem
    """
    heuristics = list(results.keys())
    domination_data = []
    
    for h1 in heuristics:
        for h2 in heuristics:
            if h1 == h2:
                continue
                
            # Match problems by solution depth for fair comparison
            h1_by_depth = defaultdict(list)
            h2_by_depth = defaultdict(list)
            
            for r in results[h1]:
                if r.solved:
                    h1_by_depth[r.solution_depth].append(r.nodes_expanded)
            for r in results[h2]:
                if r.solved:
                    h2_by_depth[r.solution_depth].append(r.nodes_expanded)
                    
            common_depths = set(h1_by_depth.keys()) & set(h2_by_depth.keys())
            if not common_depths:
                continue
                
            dominates = True
            strict_dominance = False
            
            for depth in common_depths:
                h1_nodes = np.mean(h1_by_depth[depth])
                h2_nodes = np.mean(h2_by_depth[depth])
                
                if h1_nodes > h2_nodes:
                    dominates = False
                    break
                if h1_nodes < h2_nodes:
                    strict_dominance = True
                    
            if dominates and strict_dominance:
                domination_data.append({
                    'dominating': h1,
                    'dominated': h2,
                    'common_problems': len(common_depths)
                })
    
    return pd.DataFrame(domination_data)

def save_comparison_graphs(results: Dict[str, List[SearchResult]], n: int, base_path: str):
    """Create and save comparison graphs for the results."""
    # Nodes vs Depth Plot
    plt.figure(figsize=(10, 6))
    for h_name, h_results in results.items():
        solved_results = [r for r in h_results if r.solved]
        if not solved_results:
            continue
            
        depths = [r.solution_depth for r in solved_results]
        nodes = [r.nodes_expanded for r in solved_results]
        plt.scatter(depths, nodes, label=h_name, alpha=0.5)
    
    plt.xlabel('Solution Depth')
    plt.ylabel('Nodes Expanded')
    plt.yscale('log')
    plt.legend()
    plt.title(f'{n}x{n} Puzzle: Solution Depth vs Nodes Expanded')
    plt.grid(True)
    plt.savefig(f'{base_path}/nodes_vs_depth_{n}x{n}.png')
    plt.close()
    
    # Branching Factor Plot
    plt.figure(figsize=(10, 6))
    depths = defaultdict(lambda: defaultdict(list))
    for h_name, h_results in results.items():
        for r in h_results:
            if r.solved:
                bf = branching_factor(r.nodes_expanded, r.solution_depth)
                depths[r.solution_depth][h_name].append(bf)
    
    for h_name in results.keys():
        x = sorted(depths.keys())
        y = [np.mean(depths[d][h_name]) for d in x if depths[d][h_name]]
        plt.plot(x, y, marker='o', label=h_name)
    
    plt.xlabel('Solution Depth')
    plt.ylabel('Effective Branching Factor')
    plt.title(f'{n}x{n} Puzzle: Effective Branching Factor vs Solution Depth')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{base_path}/branching_factor_{n}x{n}.png')
    plt.close()

def ensure_figures_directory(n=None):
    """Create Part1, Part2, Part3 directories if they don't exist and return the correct one for n."""
    import os
    parent_dir = "Results"
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if n == 3:
        part_dir = os.path.join(parent_dir, "Part1")
    elif n == 4:
        part_dir = os.path.join(parent_dir, "Part2")
    elif n == 5:
        part_dir = os.path.join(parent_dir, "Part3")
    else:
        part_dir = os.path.join(parent_dir, "figures")
    if not os.path.exists(part_dir):
        os.makedirs(part_dir)
    return part_dir

def run_part_1():
    # Run experiments for 8-puzzle
    """Run experiments for 8-puzzle."""
    """Run experiments for 8-puzzle."""
    print("\n=== Part 1: 8-puzzle Experiments ===")
    n = 3  # 8-puzzle is 3x3
    h_funcs = {
        'h1 (Misplaced)': h1_misplaced,
        'h2 (Manhattan)': h2_manhattan,
        'h3 (Linear Conflict)': h3_linear_conflict
    }
    
    results = run_multiple_experiments(n, num_moves=20, num_experiments=100, h_funcs=h_funcs)
    figures_dir = ensure_figures_directory(n)
    
    print("\nStatistics:")
    stats = calculate_statistics(results)
    print(stats)
    stats.to_csv(f'{figures_dir}/statistics_8puzzle.csv')
    
    print("\nDetailed Performance Comparison by Solution Depth:")
    comparison = create_and_save_comparison_table(results, n, figures_dir)
    print(comparison)
    
    print("\nHeuristic Domination Analysis:")
    domination = analyze_heuristic_domination(results)
    print(domination)
    domination.to_csv(f'{figures_dir}/domination_8puzzle.csv')
    
    save_comparison_graphs(results, n, figures_dir)
    # Export all raw results for each heuristic
    for h_name, h_results in results.items():
        pd.DataFrame([
            {
                'solved': r.solved,
                'solution_depth': r.solution_depth,
                'nodes_expanded': r.nodes_expanded,
                'runtime': r.runtime
            } for r in h_results
        ]).to_csv(f'{figures_dir}/raw_results_{h_name.replace(" ", "_").replace("(", "").replace(")", "")}_8puzzle.csv', index=False)
    # Save PNG table at the end
    save_image_table_nodes_and_bf(results, n, figures_dir)
    return results

def run_part_2():
    # Run experiments for 15-puzzle
    """Run experiments for 15-puzzle."""
    """Run experiments for 15-puzzle."""
    print("\n=== Part 2: 15-puzzle Experiments ===")
    n = 4  # 15-puzzle is 4x4
    h_funcs = {
        'h1 (Misplaced)': h1_misplaced,
        'h2 (Manhattan)': h2_manhattan,
        'h3 (Linear Conflict)': h3_linear_conflict
    }
    
    results = run_multiple_experiments(n, num_moves=20, num_experiments=100, h_funcs=h_funcs)
    figures_dir = ensure_figures_directory(n)
    
    print("\nStatistics:")
    stats = calculate_statistics(results)
    print(stats)
    stats.to_csv(f'{figures_dir}/statistics_15puzzle.csv')
    
    print("\nDetailed Performance Comparison by Solution Depth:")
    comparison = create_and_save_comparison_table(results, n, figures_dir)
    print(comparison)
    
    print("\nHeuristic Domination Analysis:")
    domination = analyze_heuristic_domination(results)
    print(domination)
    domination.to_csv(f'{figures_dir}/domination_15puzzle.csv')
    
    save_comparison_graphs(results, n, figures_dir)
    # Export all raw results for each heuristic
    for h_name, h_results in results.items():
        pd.DataFrame([
            {
                'solved': r.solved,
                'solution_depth': r.solution_depth,
                'nodes_expanded': r.nodes_expanded,
                'runtime': r.runtime
            } for r in h_results
        ]).to_csv(f'{figures_dir}/raw_results_{h_name.replace(" ", "_").replace("(", "").replace(")", "")}_15puzzle.csv', index=False)
    # Save PNG table at the end
    save_image_table_nodes_and_bf(results, n, figures_dir)
    return results

def run_part_3():
    # Run experiments for 24-puzzle
    """Run experiments for 24-puzzle."""
    """Run experiments for 24-puzzle."""
    print("\n=== Part 3: 24-puzzle Experiments ===")
    n = 5  # 24-puzzle is 5x5
    h_funcs = {
        'h1 (Misplaced)': h1_misplaced,
        'h2 (Manhattan)': h2_manhattan,
        'h3 (Linear Conflict)': h3_linear_conflict
    }
    
    results = run_multiple_experiments(n, num_moves=20, num_experiments=100, h_funcs=h_funcs)
    figures_dir = ensure_figures_directory(n)
    
    print("\nStatistics:")
    stats = calculate_statistics(results)
    print(stats)
    stats.to_csv(f'{figures_dir}/statistics_24puzzle.csv')
    
    print("\nDetailed Performance Comparison by Solution Depth:")
    comparison = create_and_save_comparison_table(results, n, figures_dir)
    print(comparison)
    
    print("\nHeuristic Domination Analysis:")
    domination = analyze_heuristic_domination(results)
    print(domination)
    domination.to_csv(f'{figures_dir}/domination_24puzzle.csv')
    
    save_comparison_graphs(results, n, figures_dir)
    # Export all raw results for each heuristic
    for h_name, h_results in results.items():
        pd.DataFrame([
            {
                'solved': r.solved,
                'solution_depth': r.solution_depth,
                'nodes_expanded': r.nodes_expanded,
                'runtime': r.runtime
            } for r in h_results
        ]).to_csv(f'{figures_dir}/raw_results_{h_name.replace(" ", "_").replace("(", "").replace(")", "")}_24puzzle.csv', index=False)
    # Save PNG table at the end
    save_image_table_nodes_and_bf(results, n, figures_dir)
    return results

def compare_all_results(results_8: dict, results_15: dict, results_24: dict):
    """Create unified comparison of all puzzle sizes."""
    print("\n=== Unified Comparison Across Puzzle Sizes ===")
    
    sizes = {3: results_8, 4: results_15, 5: results_24}
    
    for n, results in sizes.items():
        print(f"\n{n}x{n} Puzzle Statistics:")
        print(calculate_statistics(results))
    
    # Create combined visualization
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    for idx, (n, results) in enumerate(sizes.items()):
        for h_name, h_results in results.items():
            solved_results = [r for r in h_results if r.solved]
            if not solved_results:
                continue
                
            depths = [r.solution_depth for r in solved_results]
            nodes = [r.nodes_expanded for r in solved_results]
            
            axes[idx].scatter(depths, nodes, label=h_name, alpha=0.5)
        
        axes[idx].set_xlabel('Solution Depth')
        axes[idx].set_ylabel('Nodes Expanded')
        axes[idx].set_yscale('log')
        axes[idx].legend()
        axes[idx].set_title(f'{n}x{n} Puzzle')
    
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    # Run all experiments
    results_8 = run_part_1()
    results_15 = run_part_2()
    results_24 = run_part_3()
    
    # Show unified comparison
    compare_all_results(results_8, results_15, results_24)
